In [8]:
import copy
import tiktoken

from dataclasses import dataclass
from typing import Optional

@dataclass
class Message:
    """Base class for all messages"""
    content: str
    role: str

    def to_dict(self):
        return {
            "role": self.role,
            "content": self.content
        }

@dataclass
class System(Message):
    """System message"""
    content: str
    name: Optional[str] = None
    role: str = "system"

    def to_dict(self):
        if self.name is None:
            return {
                "role": self.role,
                "content": self.content
            }
        else:
            return {
                "role": self.role,
                "name": self.name,
                "content": self.content,
            }

@dataclass
class User(Message):
    """User message"""
    content: str
    name: Optional[str] = None
    role: str = "user"

@dataclass
class Assistant(Message):
    """Assistant message"""
    content: str
    role: str = "assistant"


class Conversation:
    """a conversation is a list of messages"""
    def __init__(self, model="gpt-4o"):
        self.model = model
        self.messages = []
        self.total_cost = 0

    def add_message(self, message: Message):
        """
        Add a message to the conversation
        """
        assert isinstance(message, Message)
        self.messages.append(message)

    def add_messages(self, messages: [Message]):
        """
        Add a list of messages to the conversation
        """
        for message in messages:
            assert isinstance(message, Message)
            self.messages.append(message)

    def prepend_message(self, message: Message):
        """
        Prepend a message to the conversation.
        Normally used to add system message in the beginning of the conversation
        """
        assert isinstance(message, Message)
        self.messages.insert(0, message)
    
    def remove_empty_messages(self):
        """
        Remove empty messages from the conversation
        """
        self.messages = [message for message in self.messages if message.content.strip() != ""]

    def to_openai(self) -> [dict]:
        """
        convert the conversation to a list of dict
        """
        return [message.to_dict() for message in self.messages]
        return conversation

    def get_cost(self) -> float:
        """
        Calculate the cost of the conversation
        every time called, returns the cost of the current call
        accumulated cost is stored in self.total_cost
        """
        # assume last message is generated by assistant
        prompt_text = ""
        for message in self.messages[:-1]:
            prompt_text += message.content
        completion_text = self.messages[-1].content

        encoding = tiktoken.encoding_for_model(self.model)
        prompt_tokens = encoding.encode(prompt_text)
        completion_tokens = encoding.encode(completion_text)

        price_k_tokens_prompt, price_k_tokens_completion = 0.03, 0.06
        if self.model.startswith("gpt-4o"):
            price_k_tokens_prompt = 0.005
            price_k_tokens_completion = 0.015
        elif self.model.startswith("gpt-4"):
            price_k_tokens_prompt = 0.03
            price_k_tokens_completion = 0.06
        elif self.model.startswith("gpt-3.5-turbo"):
            price_k_tokens_prompt = 0.0015
            price_k_tokens_completion = 0.002
        else:
            print(f"Unknown model {self.model}")
        cost = (len(prompt_tokens)/1000 * price_k_tokens_prompt) + \
               (len(completion_tokens)/1000 * price_k_tokens_completion)
        self.total_cost += cost
        return cost
    
    def __repr__(self) -> str:
        s = ""
        for message in self.messages:
            s += f"{message.role}: {message.content}  "
        return s

    def pretty_print(self):
        print("=====================================")
        for message in self.messages:
            print(f"{message.role}: {message.content}")
        print("=====================================")

    def deep_copy(self):
        """
        Deep copy the conversation
        """
        return copy.deepcopy(self)

In [9]:
# sample conversation
system_msg = System("You are a helpful assistant.")
user_msg = User("Hello, how are you?", "John")
assistant_msg = Assistant("I'm doing well, thank you for asking! How can I help you today?")

conversation = Conversation(model="gpt-4o")
conversation.add_messages([system_msg, user_msg, assistant_msg])
conversation.pretty_print()

system: You are a helpful assistant.
John: Hello, how are you?
assistant: I'm doing well, thank you for asking! How can I help you today?


In [10]:
conversation2 = conversation.deep_copy()
conversation2.pretty_print()

conversation2.add_message(User("Hello, how are you?", "John"))
conversation2.pretty_print()
conversation.pretty_print()

system: You are a helpful assistant.
John: Hello, how are you?
assistant: I'm doing well, thank you for asking! How can I help you today?
system: You are a helpful assistant.
John: Hello, how are you?
assistant: I'm doing well, thank you for asking! How can I help you today?
John: Hello, how are you?
system: You are a helpful assistant.
John: Hello, how are you?
assistant: I'm doing well, thank you for asking! How can I help you today?
